In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import time

In [3]:
req = requests.get("http://asianwiki.com/Main_Page")
soup = BeautifulSoup(req.content, "html.parser")
tag_list = soup.findAll("div", {"id": "slider"})[1].findAll("a")
urls = []
for tag in tag_list:
    urls.append(tag["href"])

In [7]:
data = []
mchain = {}
additional_columns = ["Network", "Episodes", "Country", "Language", "Title"]
for url in urls:
    #Bug in website
    if url == urls[117]:
        url = url[:len(urls[117]) - 1]
    
    kdrama = {"Network": "", 
            "Episodes": -1,
            "Country": "", 
            "Language": "", 
             "Title" : ""}
    req = requests.get(url)
    soup  = BeautifulSoup(req.content, "html.parser")
    content = soup.find("div", {"id":"mw-content-text"})

    #Rating
    rating_content = content.findAll("b")
    try:
        kdrama["rating"] = float(rating_content[0].text.split("/")[0].rstrip().lstrip())
    except (ValueError, IndexError) as e:
        kdrama["rating"] = float('NaN')

    #Comments
    comments = []
    comments_char_len = 0
    comments_word_len = 0
    for div in content.findAll("div", {"class":"comment-item"}):
        comment = div.text.replace('\xa0', "").split('\n')[2]
        comments.append(comment)
        comments_char_len += len(comment)
        comments_word_len += len(comment.split(" "))
    kdrama["comments"] = " ".join(comments)
    kdrama["no_comments"] = len(comments)
    if len(comments) != 0:
        kdrama["avg_comment_char_len"] = comments_char_len / len(comments)
        kdrama["avg_comment_word_len"] = comments_word_len / len(comments)
    else:
        kdrama["avg_comment_char_len"] = 0
        kdrama["avg_comment_word_len"] = 0
    if kdrama["rating"] not in mchain:
        mchain[kdrama["rating"]] = comments
    else: 
        mchain[kdrama["rating"]] += comments
    
    #Title
    titles = soup.findAll("h1")
    if len(titles) >= 1:
        kdrama["Title"] = titles[0].get_text()
    
    #Network, Country, Language, Episodes
    for ils in soup.findAll("li"):
        text = ils.get_text()
        components = text.split(":")
        if components[0].strip() in additional_columns: 
            kdrama[components[0].strip()] = components[1].strip()
    
    data.append(kdrama)
    time.sleep(0.25)

In [8]:
drama_df = pd.DataFrame(data)
drama_df.to_csv("./dramas.csv")
drama_df.head()

,Country,Episodes,Language,Network,Title,avg_comment_char_len,avg_comment_word_len,comments,no_comments,rating
0,Japan,10,Japanese,"TV Asahi, ABC",A Little Room For Hope,101.666667,19.333333,interesting to watch...love the plot and only ...,3,90.0
1,Japan,2,Japanese,"NTV, YTV",Manager Rinko Kazehana's Love,0.000000,0.000000,,0,43.0
2,South Korea,24,Korean,tvN,Mr. Sunshine,168.477449,31.181960,"this is epic! so good i cried. the plot, the c...",643,85.0
3,South Korea,32,Korean,KBS2,Your House Helper,180.960396,33.158416,This has to be my favorite role that Ha Seok-J...,101,86.0
4,Japan,10,Japanese,Fuji TV,Absolute Zero 3,167.000000,30.000000,Well i hope this drama will continue for new s...,1,84.0


In [13]:
import pickle
pickle.dump(mchain, open("mchain.pkl", "wb"))